In [1]:
import numpy as np
lambdas = np.array([[1], [2]])
weightss = np.array([
    [[0,2,3,4],[1,2,3,4]], 
    [[0,2,3,4],[1,2,3,4]]],
      dtype=np.float32)


# LUT, K x n x 3 x 3


In [16]:
import numpy as np

def broadcasted_multiply(arrays, lambdax):
    dims = len(arrays)  # số chiều cần broadcast

    reshaped = []
    for i, arr in enumerate(arrays):
        arr_np = np.array(arr)
        # reshape thành (1, 1, ..., len(arr), ..., 1)
        shape = [1] * dims
        shape[i] = arr_np.shape[0]
        reshaped.append(arr_np.reshape(shape))

    # Nhân phân phối
    result = reshaped[0]
    for r in reshaped[1:]:
        result = result * r

    return lambdax*result.flatten()

def broadcasted_multiplies(arrayss, lambdas):
	result = []
	for i, arrays in enumerate(arrayss):
		result.append(broadcasted_multiply(arrays, lambdas[i]))
	return result

# Ví dụ sử dụng
lambdas = np.array([1] * 10000, dtype=np.float32)
arrayss = [
    [np.array([2, 1, 1], dtype=np.float32), 
     np.array([1.2], dtype=np.float32), 
     np.array([1, 2], dtype=np.float32), 
     np.array([3, 4], dtype=np.float32)
     ] for i in range(10000)
]

k = broadcasted_multiplies(arrayss, lambdas)

In [11]:
import numpy as np

def encode_base4(arrays):
    n = len(arrays)
    reshaped = []
    
    # reshape từng array để broadcasting được
    for i, arr in enumerate(arrays):
        arr_np = np.array(arr, dtype=int)
        shape = [1] * n
        shape[i] = -1  # gán chiều tương ứng
        reshaped.append(arr_np.reshape(shape))

    # tạo weights: 4^(n-1), 4^(n-2), ..., 4^0
    powers = [4 ** (n - i - 1) for i in range(n)]

    result = reshaped[0] * powers[0]
    for i in range(1, n):
        result = result + reshaped[i] * powers[i]

    return result.flatten()

def encode_base4s(arrayss):
	result = []
	for arrays in arrayss:
		result.append(encode_base4(arrays))
	return result

arrayss = [
    [[2, 1, 1], [1], [1, 2]],
    [[2, 2, 1], [0], [1]],
]

result = encode_base4s(arrayss)
print(result)


[array([37, 38, 21, 22, 21, 22]), array([33, 33, 17])]


In [4]:
import numpy as np

def sum_distributions(list_of_value_pos_pairs):
    all_positions = []
    all_values = []

    for values, positions in list_of_value_pos_pairs:
        all_positions.append(np.array(positions))
        all_values.append(np.array(values))

    all_positions = np.concatenate(all_positions)
    all_values = np.concatenate(all_values)

    # Tính max index cần thiết cho bincount
    max_pos = all_positions.max()
    summed = np.bincount(all_positions, weights=all_values)

    # Chỉ lấy các position có giá trị tổng khác 0
    non_zero_pos = np.nonzero(summed)[0]
    non_zero_values = summed[non_zero_pos]

    return non_zero_pos, non_zero_values
# List 1
values1 = [10, 20, 1]
positions1 = [1, 2, 3]

# List 2
values2 = [5, -15]
positions2 = [2, 3]  # trùng với list1 → sẽ được cộng dồn

# List 3
values3 = [100]
positions3 = [10]

# Tổng hợp
all_lists = [
    (values1, positions1),
    (values2, positions2),
    (values3, positions3)
]

positions, totals = sum_distributions(all_lists)
print("Positions:", positions)
print("Total values:", totals)


Positions: [ 1  2  3 10]
Total values: [ 10.  25. -14. 100.]
